In [ ]:
#CHANGE THE FILE NAME

import numpy as np
import pandas as pd
from cellxgene_mods import CxG_API
from public_resources import *

# Search public resources

In [ ]:
#Figshare searches may return too many results - unclear how to do phrase searching (e.g. "Paired-seq")
fshare_df = figshare_query(search_term)
fshare_df

In [ ]:
bs_df = biostudies_query(search_term)
bs_df

In [ ]:
geo_df = geo_query(search_term)
geo_df

In [ ]:
z_df = zenodo_query(search_term)
z_df

**Merge**

In [ ]:
add_dois = {}

merged_df = pd.concat([bs_df, geo_df,fshare_df, z_df])
unfound_accs = [k for k in add_dois.keys() if k not in merged_df['accession'].unique()]
if unfound_accs:
    print('accession not in dataframe')
    print(unfound_accs)

In [ ]:
merged_df['doi'] = merged_df.apply(lambda x: add_dois.get(x['accession'], x['doi']), axis=1)

for pmid in merged_df[(merged_df['doi'].isna()) & (merged_df['pmid'].isna() == False)]['pmid'].dropna().unique():
    if pmid:
        doi = pmid_to_doi(pmid)
        merged_df.loc[merged_df['pmid'] == pmid, 'doi'] = doi

merged_df.groupby('doi').agg({
    'accession': lambda x: ','.join(str(x)),
    'pmid': lambda x: ','.join(str(x)),
    'title': lambda x: ','.join(str(x)),
    'file formats': lambda x: ','.join([e for e in x]),
    'organism': lambda x: ','.join(str(x))
})

pub_info = []
for doi in merged_df['doi'].dropna().unique():
    if doi:
        pub_info.append(doi_checker(doi))

pub_df = pd.DataFrame(pub_info)

merged_df = merged_df.merge(pub_df, left_on='doi', right_on='input_doi', how='left', suffixes=('_x',None))
merged_df['Title'] = merged_df.apply(lambda x: x['title'] if pd.isnull(x['doi']) else x['Title'], axis=1)
remove = ['input_doi','doi_x','invalid DOI','pmid','title']
merged_df.drop(columns=[c for c in remove if c in merged_df.columns], inplace=True)
merged_df.fillna({'file formats': ''}, inplace=True)

pmid_dois = publication_search(search_term)
pub_df = pd.DataFrame.from_dict(pmid_dois, orient='index', columns=['doi']).reset_index().rename(columns={'index':'pmid'})
pub_df = pub_df[pub_df['doi'].isin(merged_df['doi']) == False]
merged_df = pd.concat([merged_df,pub_df])

merged_df.to_csv(f'{search_term}_records.csv', index=False)

# Identify publications

In [ ]:
dois = publication_search(search_term)

new_pub_info = []
for doi in dois:
    if doi and doi not in merged_df['doi'].unique():
        new_pub_info.append(doi_checker(doi))
new_pub_df = pd.DataFrame(new_pub_info)
new_pub_df.drop(columns=['input_doi','invalid DOI'], inplace=True)
new_pub_df = new_pub_df[new_pub_df['doi'].isin(merged_df['doi'].unique()) == False]

all_pubs = pd.concat([merged_df, new_pub_df])

all_pubs.to_csv(f'{search_term}_publications.csv', index=False)
all_pubs